### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
# %pip install semantic-link-labs
%pip install "builtin/semantic_link_labs-0.9.1-py3-none-any.whl"

### Requirements
* Fabric Capacity with XMLA read/write enabled
    * A Fabric Trial Capacity is sufficient for evaluation.
    * The XMLA Endpoint must be read/write enabled because the perf lab provisions semantic models automatically.
* Fabric Permissions
    * User must have permissions to create workspaces, lakehouses, and semantic models. This notebook provisions sample resources to demonstrate the use of a perf lab.
    * User should have access to a Fabric capacity. This notebook provisions workspaces, lakehouses, and semantic models on a Fabric capacity.
    * Connect this notebook to a lakehouse without a schema to persist test definitions and test results. Although strictly not a requirement, it eliminates the need to provide the name and Id of a disconnected lakehouse.

### Result
* A master and test workspaces, lakehouses, and semantic models are created to establish a perf lab
    * The master workspace contains a lakehouse and a sample semantic model in Direct Lake on OneLake mode that uses the lakehouse as its data source. 
    * The test workspace contains semantic models cloned from the sample semantic model in the master workspace.
    * Various Delta tables are created in the lakehouse connected to this notebook to persist test definitions, table analysis, and test results.
    * The resources in the master workspace and in the test workspace are deprovisioned upon completion of the perf lab. Delete the workspaces manually.
* The names of the newly created resources can be adjusted to customize the perf lab.


### Import the library and set global notebook parameters

This notebook deploys lakehouses and semantic models across different workspaces, but the resources can also be hosted together in a centralized workspace. The master workspace contains a lakehouse with sample data, used as the data source for the sample semantic models in Direct Lake on OneLake mode. The master semantic model serves as a template for the actual test models, which this notebook provisions prior to running the performance tests by cloning the master semantic model.

In [ ]:
import sempy_labs.perf_lab as perf_lab


master_workspace = 'Perf Lab Master'                # Enter the name of the master workspace.
lakehouse = 'SampleLakehouse'                       # Enter the name of the lakehouse used as the data source.
master_dataset = 'Master Semantic Model'            # Enter the name of the master semantic model.

test_workspace = 'Perf Lab Testing'                 # Enter the name of the workspace for the semantic model clone.
target_dataset_prefix = 'Test Model_'               # Enter the common part of the name for all semantic model clones.
test_dataset_A = target_dataset_prefix + 'A'        # Enter the name of the first semantic model clone.
test_dataset_B = target_dataset_prefix + 'B'        # Enter the name of the second semantic model clone.

capacity_id = None                                  # The Id of the capacity for the workspaces. 
                                                    #Leave this as None to use the capacity of the attached lakehouse or perf lab notebook.
                                        
test_definitions_tbl = 'TestDefinitions'            # The name of the table in the notebook-attached lakehouse to store the test definitions.
column_segments_tbl = 'StorageTableColumnSegments'  # The name of the table in the notebook-attached lakehouse to store the test definitions.
trace_events_tbl = "TraceEvents"                    # The name of the table in the notebook-attached lakehouse to store the captured trace events.

### Provision master workspace, lakehouse, and semantic model with sample data
A sample lakehouse can be provisioned by calling the provision_perf_lab_lakehouse() function with the provision_sample_semantic_model() table-generator functions. If you want to customize the table generation, use the source code for the _get_sample_tables_property_bag() and provision_sample_delta_tables() functions as a starting point. The sample semantic model uses the sample lakehouse, but you can bring your own semantic model and set the name and id accordingly.

In [ ]:
# Create a workspace and a lakehouse.
# Keep track of the workspace_id and lakehouse_id for subsequent function calls.

(master_workspace_id, lakehouse_id) = perf_lab.provision_perf_lab_lakehouse(
    workspace = master_workspace, 
    lakehouse = lakehouse,
    table_properties=perf_lab._get_sample_tables_property_bag(fact_rows_in_millions = 10),
    table_generator=perf_lab.provision_sample_delta_tables,
    capacity_id = capacity_id
) 

# Create a master semantic model.
# Keep track of name and Id for subsequent function calls.
(master_dataset_name, master_dataset_id) = perf_lab.provision_sample_semantic_model(
    workspace = master_workspace_id, 
    lakehouse=lakehouse_id, 
    semantic_model_name = master_dataset
    )

### Generate sample test definitions using sample queries
The sample test definition leverage some predefined sample queries that work with the sample semantic models provisioned by using the provision_sample_semantic_model() function. You can also use the _get_test_definitions_from_trace_events() function to generate the test definitions from a Profiler trace as demonstrated in a subsequent cell. 

In [ ]:
# _sample_queries is a dictionary with a query name or Id as the key and the query text as the value.
# The _get_test_definitions() functions generates test definitions to execute all _sample_queries against the same test semantic model.

test_definitions_A = perf_lab._get_test_definitions(
    dax_queries = perf_lab._sample_queries, 
    target_dataset = test_dataset_A,
    target_workspace = test_workspace,
    master_dataset = master_dataset_name,
    master_workspace = master_workspace_id,
    data_source = lakehouse,
    data_source_workspace = master_workspace_id,
    )

test_definitions_B = perf_lab._get_test_definitions(
    dax_queries = perf_lab._sample_queries, 
    target_dataset = test_dataset_B,
    target_workspace = test_workspace,
    master_dataset = master_dataset_name,
    master_workspace = master_workspace_id,
    data_source = lakehouse,
    data_source_workspace = master_workspace_id,
    )

# Presisting the results in the notebook-attached lakehouse so that the definitions can easily be retrieved for subsequent runs.
# Note that the _save_as_delta_table() functions overwrites any existing test definitions table.

perf_lab._save_as_delta_table(
        dataframe = test_definitions_A.union(test_definitions_B),
        delta_table_name = test_definitions_tbl
        )

### Generate sample test definitions from trace events
As an alternative to the cell above, you can run the _get_test_definitions_from_trace_events() function and interact with the master semantic model using Power BI Desktop in LiveConnect mode or other client tools or reports to generate the test definitions from a Profiler trace. The _get_test_definitions_from_trace_events() function will run in a loop until a customizable timeout expires (5 minutes by default) or until you send an EVALUATE {"Stop"} DAX query to the master model.

In [ ]:
# The _get_test_definitions_from_trace_events() function generates test definitions from trace events against the master model
# and associates each query with a separate test semantic model.
# Start the trace collection by running the _get_test_definitions_from_trace_events() function, then interact with the model to send DAX queries.
# Wait for the timeout to expire to end the trace collection or send an EVALUATE {"Stop"} DAX query to exist the trace collection sooner.

traced_definitions = perf_lab._get_test_definitions_from_trace_events(
    target_dataset_prefix = target_dataset_prefix,
    target_workspace = test_workspace,
    master_dataset = master_dataset_name,
    master_workspace = master_workspace_id,
    data_source = lakehouse,
    data_source_workspace = master_workspace_id,
    timeout = 300
    )

# Presist the results in the notebook-attached lakehouse so that the definitions can easily be retrieved for subsequent runs.
# Note that the _save_as_delta_table() functions overwrites any existing test definitions table.

perf_lab._save_as_delta_table(
        dataframe = traced_definitions,
        delta_table_name = test_definitions_tbl
        )


### Provision test semantic models
Creating numerous semantic models for testing can easily be accomplished by passing the spark dataframe with the test definitions to the _provision_test_models() function. For every unique combination of 'MasterWorkspace', 'MasterDataset', 'TargetWorkspace', and 'TargetDataset', this function creates the necessary semantic model clones that the test cycles later use to run DAX queries.

In [ ]:
# Load the test definitions from the notebook-attached lakehouse.

test_definitions = perf_lab._read_delta_table(
    delta_table_name = test_definitions_tbl
    )

# Provision the test models by cloning the master semantic models
# in the specified test workspaces according to the test definitions.

perf_lab._provision_test_models( 
    test_definitions = test_definitions,
    capacity_id = capacity_id,
    refresh_clone = True,
    )

### Prepare a test cycle
Text cycle-specific information includes a test run Id and a timestamp, which must be passed along with the test definition so that the text cycle-specific Id and timestamp can be included in the test results for later analysis of individual test runs.

In [ ]:
# Add a test run Id and a timestamp to all test definitions.

test_cycle_definitions = perf_lab._initialize_test_cycle(
    test_definitions = test_definitions
    )

display(test_cycle_definitions)

### Warm up the test models
Before updating Delta tables and refreshing Direct Lake models, it is a good idea to simulate semantic models that are currently in use by running all the test queries without tracing. This brings the test semantic models into warm state.

In [ ]:
# Execute all queries in test definitions against their test models
# so that all relevant column data is loaded into memory.

perf_lab._warmup_test_models(
    test_definitions = test_cycle_definitions
) 

### Simulate Lakehouse ETL
The perf lab has no real ETL pipeline and must therefore rely on a simulated ETL process. The perf lab accomplishes the work with the help of a sample callback function. Refer to the source code if you want to implement your own table update logic.

In [ ]:
# To update Delta tables, determine the list of Delta tables that must be processed. 

source_table_props = perf_lab.PropertyBag()
source_table_props.add_property("Prefix", "sales")

table_info = perf_lab.get_source_tables(
    test_definitions = test_cycle_definitions,
    filter_properties = source_table_props,
    filter_function = perf_lab._filter_by_prefix
    )

# Use the sample _filter_by_prefix() callback function 
# to perform a rolling window update by deleting the oldest DateID
# and reinserting it as the newest DateID.
# The _filter_by_prefix() callback function expects a property bag
# that identifies the DateID column as the key column.
delete_reinsert_props = perf_lab.PropertyBag()
delete_reinsert_props.add_property("key_column", "DateID")

perf_lab.simulate_etl(
    source_tables_info = table_info,
    update_properties = delete_reinsert_props,
    update_function = perf_lab._delete_reinsert_rows
)

### Analyze Delta tables and semantic model tables
To investigate the dependencies and interactions between Delta tables and Direct Lake models in various configurations, the perf lab includes functions to analyze the column segments for each table in the semantic model as well as the parquet files, storage groups, and other information for the Delta tables.

In [ ]:
# Under the covers, calls the INFO.STORAGETABLECOLUMNSEGMENTS DAX function 
# to retrieve details about the column segments for all model tables listed in tables_info.

column_segments = perf_lab.get_storage_table_column_segments(
    test_cycle_definitions = test_cycle_definitions,
    tables_info = table_info
    ) 

# Insert the results in the notebook-attached lakehouse for later analysis.

perf_lab._insert_into_delta_table(
        dataframe = column_segments,
        delta_table_name = column_segments_tbl
        )

### Run default (incremental), cold, and warm query tests
The main purpose of a test run is to measure the performance of a set of DAX queries against the test semantic models with different memory states: Cold (full framing), Semi-warm (incremental framing), and Warm (no framing). Other than running the queries and measuring response times, the run_test_cycle() function must therefore perform additional actions, specifically clearing the cache and refreshing the model.

In [ ]:
# Lakehouse ETL was simulated earlier. The Delta tables are updated.
# Performing a full refresh triggers incremental framing to reload 
# only the data that was impacted by the latest updates.

inc_results = perf_lab.run_test_cycle(
    test_cycle_definitions = test_cycle_definitions,
    clear_query_cache = True,
    refresh_type = "full",
    tag = "incremental"
    )

# In order to compare query perf after incremental framing with
# truly cold query performance, it is necessary to perform a
# clearValues refresh first and then a full refresh. Now,
# all of the column data must be loaded again from the Delta tables.

cold_results = perf_lab.run_test_cycle(
    test_cycle_definitions = test_cycle_definitions,
    clear_query_cache = True,
    refresh_type = "clearValuesFull",
    tag = "cold"
    )

# A warm test run is a run without refreshing the model.

warm_results = perf_lab.run_test_cycle(
    test_cycle_definitions = test_cycle_definitions,
    clear_query_cache = True,
    refresh_type = None,
    tag = "warm"
    )

# The run_test_cycle() functions returns a tuple of a Spark DataFrame with the
# captured trace events and a dictionary with the query results. We are only
# interested in the trace events. Combine them in one DataFrame and insert them
# into a Delta table in the notebook-attached lakehouse.

all_trace_events = inc_results[0].union(cold_results[0]).union(warm_results[0])

perf_lab._insert_into_delta_table(
        dataframe = column_segments,
        delta_table_name = trace_events_tbl
        )

### Deprovision perf lab resources
The perf lab also provides functions to clean up provisioned resources. However, the perf lab does not delete workspaces to avoid accidental data loss if an existing workspace with unrelated items was used in the perf lab. Note that deprovisioning lakehouses listed in the test definitions does indeed remove these lakehouses with all their tables. Make sure these lakehouses only contain perf lab tables or delete the resources manually.

In [ ]:
# Delete the master and test models listed in the test definitions.
# Set masters_and_clones = False if you only want to delete the test models
# but want to keep the master semantic model(s). 

perf_lab.deprovision_perf_lab_models(
    test_definitions = test_cycle_definitions,
    masters_and_clones = True
    )

# Delete the lakehouses listed in the test definitions.

perf_lab.deprovision_perf_lab_lakehouses(
    test_definitions = test_cycle_definitions
    )
